# Xây dựng mô hình Neural Network

- Mục tiêu: xây dựng mô hình Neural Network để phân loại ảnh cho bộ dữ liệu FashionMNIST.




In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 1. GPU hay CPU
- Kiểm tra máy có GPU để tăng tốc huấn luyện không, nếu không có thì dùng CPU


In [7]:
# INSERT CODE HERE
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## 2. Tóm tắt lý thuyết
- Cần: $ f(x) \rightarrow y $, $ x $ là input ảnh $28\times28$, $ y $ là output nhãn lớp.
- Min khoảng cách $ D(f(x), y) $
- Neural Network là một cách biểu diễn $ f $ sao cho $ D $ có đạo hàm.

<div>
<img src="nn.png" width="500"/>
</div>

## 3. Định nghĩa lớp NeuralNetwork
- Kế thừa `nn.Module`, khởi tạo ở `__init__()`, định nghĩa `f(x)` ở hàm `forward`




In [9]:
# INSERT CODE HERE
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(28*28, 512)
        self.hidden1 = nn.ReLU()
        self.layer2 = nn.Linear(512, 512)
        self.hidden2 = nn.ReLU()
        self.output_layer = nn.Linear(512, 10)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = self.hidden1(x)
        x = self.layer2(x)
        x = self.hidden2(x)
        out = self.output_layer(x)
        return out

- Khởi tạo 1 đối tượng ``NeuralNetwork`` trên ``device``, in ra cấu trúc của nó



In [10]:
# INSERT CODE HERE
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layer1): Linear(in_features=784, out_features=512, bias=True)
  (hidden1): ReLU()
  (layer2): Linear(in_features=512, out_features=512, bias=True)
  (hidden2): ReLU()
  (output_layer): Linear(in_features=512, out_features=10, bias=True)
)


- Sử dụng mô hình `f`: gọi model(x) --> gọi đến hàm foward. **Chú ý:** không gọi trực tiếp ``model.forward()``


In [13]:
# INSERT CODE HERE
X = torch.rand(1, 28, 28, device=device)
out = model(X)
print(out)
pred_prob = nn.Softmax(dim=1)(out)
print(pred_prob)
y_pred = pred_prob.argmax(1)
print("Prediction: {}".format(y_pred))

tensor([[-0.0022, -0.0129,  0.0036,  0.0089,  0.0578,  0.0318, -0.0480,  0.0501,
         -0.0328,  0.0304]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[0.0989, 0.0978, 0.0994, 0.1000, 0.1050, 0.1023, 0.0944, 0.1042, 0.0959,
         0.1021]], device='cuda:0', grad_fn=<SoftmaxBackward>)
Prediction: tensor([4], device='cuda:0')


--------------




## 4. Tham số mô hình
- Các layer là các hàm với tham số tương ứng, ta cần tìm bộ tham số để tối ưu khoảng cách đã nói ở trên.
- Tham số mô hình của NN được gọi là weights

In [15]:
# INSERT CODE HERE
print(model)

for name, param in model.named_parameters():
    print("Layer {} | Size {} | Values {}".format(name, param.size(), param[:2]))

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layer1): Linear(in_features=784, out_features=512, bias=True)
  (hidden1): ReLU()
  (layer2): Linear(in_features=512, out_features=512, bias=True)
  (hidden2): ReLU()
  (output_layer): Linear(in_features=512, out_features=10, bias=True)
)
Layer layer1.weight | Size torch.Size([512, 784]) | Values tensor([[ 0.0065, -0.0152,  0.0235,  ..., -0.0170,  0.0135, -0.0188],
        [ 0.0083, -0.0191,  0.0325,  ...,  0.0164, -0.0079, -0.0271]],
       device='cuda:0', grad_fn=<SliceBackward>)
Layer layer1.bias | Size torch.Size([512]) | Values tensor([0.0332, 0.0264], device='cuda:0', grad_fn=<SliceBackward>)
Layer layer2.weight | Size torch.Size([512, 512]) | Values tensor([[-0.0155,  0.0013, -0.0212,  ..., -0.0172,  0.0082,  0.0199],
        [-0.0137, -0.0216, -0.0196,  ..., -0.0424,  0.0050, -0.0171]],
       device='cuda:0', grad_fn=<SliceBackward>)
Layer layer2.bias | Size torch.Size([512]) | Values tensor([ 0.0216, -0.0423], d